In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_supra_fresh.d_fnr_fresh_delivery_schedule
-- CLUSTER BY city 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH
TEMP_0 AS (
    SELECT 
        city,
        udp_wcm_dev.default.n_fnr_fresh_order_num_in_str(delivery_schedule) delivery_schedule,
        send_fc_date,
        divide_po_date,
        self_life_adjust
    FROM {catalog_name}.udp_wcm_silver_dataportal.fnr_fresh_order_config_delivery_schedule
    WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_dataportal.fnr_fresh_order_config_delivery_schedule)
),
TEMP_1 AS (
    SELECT
        city,
        delivery_schedule,
        CASE 
            WHEN send_fc_date = 'D' THEN 0
            ELSE CAST(SUBSTR(send_fc_date, 3, 1) AS BIGINT)                      
        END send_fc_date,
        CASE 
            WHEN divide_po_date = 'D' THEN 0
            ELSE CAST(SUBSTR(divide_po_date, 3, 1) AS BIGINT)                      
        END divide_po_date,
        CASE 
            WHEN self_life_adjust = 'D' THEN 0
            ELSE CAST(SUBSTR(self_life_adjust, 3, 1) AS BIGINT) * (-1)                     
        END self_life_adjust,
        udp_wcm_dev.default.fn_fnr_fresh_get_order_schedule(delivery_schedule, send_fc_date) order_schedule
    FROM TEMP_0
),
TEMP_2 AS (
    SELECT 
        city,
        order_schedule,
        CASE 
            WHEN array_contains(split(a.order_schedule, ','), CAST(dayofweek(current_date()) AS STRING)) 
            THEN 'YES' 
            ELSE 'NO' 
        END AS is_order_date,
        a.delivery_schedule,
        a.send_fc_date,
        current_date() today,
        date_add(current_date(), CAST(a.send_fc_date AS INT)) first_delivery_day
    FROM TEMP_1 a
),
TEMP_3 AS (
    SELECT 
        f.city,
        f.order_schedule,
        f.is_order_date,
        f.delivery_schedule,
        f.send_fc_date,
        f.today,
        f.first_delivery_day,
        sort_array(collect_list(g.calday))[0] AS second_delivery_day,
        sort_array(collect_list(g.calday))[1] AS second_delivery_day_ifsunday
    FROM TEMP_2 f
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_time g
           ON instr(concat(',', f.delivery_schedule, ','), concat(',', g.dayofweek, ',')) > 0
    WHERE g.calday > f.first_delivery_day
    GROUP BY 1,2,3,4,5,6,7
),
MAIN AS (
    SELECT 
        city,
        order_schedule,
        is_order_date,
        delivery_schedule,
        send_fc_date,
        today,
        first_delivery_day,
        second_delivery_day
        -- CASE 
        --     WHEN dayofweek(second_delivery_day) = 1 
        --     THEN second_delivery_day_ifsunday 
        --     ELSE second_delivery_day 
        -- END AS second_delivery_day
    FROM TEMP_3
)
SELECT * FROM MAIN
""")